# 환경 준비

In [1]:
# colab에 Mecab설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [2]:
%cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [3]:
!bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 56.3 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-08-09 04:08:46--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::34cc:ea4a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=REtEu0QiyhJtD5ZdXu6T7TIbgaQ%3D&E

# 모델 실행 코드

In [4]:
# 필요 프레임워크 및 라이브러리 준비
import re, os, json
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [5]:
# 가상환경에서 GPU사용불가로 인한 오류메시지 숨김
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

In [6]:
# 서버 메모리 최적화
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [11]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP_Classification/sentiment/Deploy Version/v2.0

/content/drive/MyDrive/Colab Notebooks/NLP_Classification/sentiment/Deploy Version/v2.0


In [12]:
# 분류모델 가중치 파일 로드
loaded_model = load_model('sentiment_model_v2.0.h5')

# 단어집합별 성능 테스트

In [13]:
# 토그나이징 & 불용어정의& 보캡사이즈 정의 & 패딩길이정의
mecab = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
vocab_size = 8117 
max_len = 40 
tokenizer = Tokenizer(vocab_size, oov_token ='OOV')

In [14]:
# 사전준비된 단어집합(정수인코딩) 파일 로드
with open('sentiment_indexed_vocab_v2.0.json', encoding='UTF8') as json_file:
    tokenizer.word_index = json.load(json_file)

In [61]:
# 전처리 함수
def sentiment_preprocessing(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', new_sentence)
    if not new_sentence or new_sentence.isspace():    # 공백이거나 NUll 인 경우
        return -1
    else :                                            # 공백이거나 NUll 인 아닌 경우
        new_sentence = mecab.morphs(new_sentence) # 토큰화,   "stem =True" 어간 추출(ex 해야지 -> 하다)
        new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    
    if not new_sentence : # 불용어 제거 후 빈리스트 인 경우
        return -1
    else:
        encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
        encoded = [list(filter(bool, encoded[0]))] # 이중리스트는 필터 안먹음
        padded = pad_sequences(encoded, maxlen = max_len) # 패딩
        return padded

In [63]:
# 예측 함수
def sentiment_predict(padded_sentence):
    score = loaded_model.predict(padded_sentence)
    return float(score)

In [81]:
def sentiment_model(new_sentence):
    padded_sentence = sentiment_preprocessing(new_sentence)
    if isinstance(padded_sentence, int) :
        score = float(-1)     # 프론트에서 계산에 미포함되도록 (0-1)구간 예외값 리턴
    else :
        score = sentiment_predict(padded_sentence)
    # 코랩/주피터에서 동작 테스트 코드
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))
    return score

In [82]:
sentiment_model('')

200.00% 확률로 부정 리뷰입니다.



-1.0

In [83]:
sentiment_model('노잼 ..완전 재미 없음 ㅉㅉ')

99.68% 확률로 부정 리뷰입니다.



0.003220653161406517

In [84]:
sentiment_model('조금 어렵지만 재밌음ㅋㅋ')

98.18% 확률로 긍정 리뷰입니다.



0.9818167686462402

In [85]:
sentiment_model('케릭터가 예뻐서 좋아요')

90.23% 확률로 긍정 리뷰입니다.



0.9022966027259827

In [86]:
sentiment_model('이 상품 진짜 좋아요... 저는 강추합니다. 대박')

97.93% 확률로 긍정 리뷰입니다.



0.979292631149292

In [87]:
sentiment_model('진짜 배송도 늦고 개짜증나네요. 뭐 이런 걸 상품이라고 만듬?') #ok

99.88% 확률로 부정 리뷰입니다.



0.0012046127812936902

In [88]:
sentiment_model('판매자님... 너무 짱이에요.. 대박나삼')#ok

96.34% 확률로 긍정 리뷰입니다.



0.9633856415748596

In [ ]:
sentiment_model('ㅁㄴㅇㄻㄴㅇㄻㄴㅇ리뷰쓰기도 귀찮아') #ok

87.84% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_model('이 영화 개꿀잼 ㅋㅋㅋ') #ok

99.13% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_model('이 영화 핵노잼 ㅠㅠ') #ok

96.05% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_model('이딴게 영화냐 ㅉㅉ') #ok

99.82% 확률로 부정 리뷰입니다.

